# Operaciones avanzadas con DataFrames

## Descripción de las variables

El dataset, obtenido de <a target = "_blank" href="https://www.transtats.bts.gov/Fields.asp?Table_ID=236">este link</a> está compuesto por las siguientes variables referidas siempre al año 2018:

1. **Month** 1-4
2. **DayofMonth** 1-31
3. **DayOfWeek** 1 (Monday) - 7 (Sunday)
4. **FlightDate** fecha del vuelo
5. **Origin** código IATA del aeropuerto de origen
6. **OriginCity** ciudad donde está el aeropuerto de origen
7. **Dest** código IATA del aeropuerto de destino
8. **DestCity** ciudad donde está el aeropuerto de destino  
9. **DepTime** hora real de salida (local, hhmm)
10. **DepDelay** retraso a la salida, en minutos
11. **ArrTime** hora real de llegada (local, hhmm)
12. **ArrDelay** retraso a la llegada, en minutos: se considera que un vuelo ha llegado "on time" si aterrizó menos de 15 minutos más tarde de la hora prevista en el Computerized Reservations Systems (CRS).
13. **Cancelled** si el vuelo fue cancelado (1 = sí, 0 = no)
14. **CancellationCode** razón de cancelación (A = aparato, B = tiempo atmosférico, C = NAS, D = seguridad)
15. **Diverted** si el vuelo ha sido desviado (1 = sí, 0 = no)
16. **ActualElapsedTime** tiempo real invertido en el vuelo
17. **AirTime** en minutos
18. **Distance** en millas
19. **CarrierDelay** en minutos: El retraso del transportista está bajo el control del transportista aéreo. Ejemplos de sucesos que pueden determinar el retraso del transportista son: limpieza de la aeronave, daño de la aeronave, espera de la llegada de los pasajeros o la tripulación de conexión, equipaje, impacto de un pájaro, carga de equipaje, servicio de comidas, computadora, equipo del transportista, problemas legales de la tripulación (descanso del piloto o acompañante) , daños por mercancías peligrosas, inspección de ingeniería, abastecimiento de combustible, pasajeros discapacitados, tripulación retrasada, servicio de inodoros, mantenimiento, ventas excesivas, servicio de agua potable, denegación de viaje a pasajeros en mal estado, proceso de embarque muy lento, equipaje de mano no válido, retrasos de peso y equilibrio.
20. **WeatherDelay** en minutos: causado por condiciones atmosféricas extremas o peligrosas, previstas o que se han manifestado antes del despegue, durante el viaje, o a la llegada.
21. **NASDelay** en minutos: retraso causado por el National Airspace System (NAS) por motivos como condiciones meteorológicas (perjudiciales pero no extremas), operaciones del aeropuerto, mucho tráfico aéreo, problemas con los controladores aéreos, etc.
22. **SecurityDelay** en minutos: causado por la evacuación de una terminal, re-embarque de un avión debido a brechas en la seguridad, fallos en dispositivos del control de seguridad, colas demasiado largas en el control de seguridad, etc.
23. **LateAircraftDelay** en minutos: debido al propio retraso del avión al llegar, problemas para conseguir aterrizar en un aeropuerto a una hora más tardía de la que estaba prevista.

In [35]:
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType

# Leemos los datos y quitamos filas con NA y convertimos a numéricas las columnas inferidas incorrectamente
flightsDF = spark.read\
                 .option("header", "true")\
                 .option("inferSchema", "true")\
                 .csv("C:/Users/nerea.gomez/Documents/Documentacion/Ejercicios DataFrames/flights-jan-apr-2018.csv")

# Convertimos a enteros y re-categorizamos ArrDelay en una nueva columna ArrDelayCat
# None (< 15 min), Slight(entre 15 y 60 min), Huge (> 60 min)

cleanFlightsDF = flightsDF.withColumn("ArrDelayCat", F.when(F.col("ArrDelay") < 15, "None")\
                                                      .when((F.col("ArrDelay") >= 15) & (F.col("ArrDelay") < 60), "Slight")\
                                                      .otherwise("Huge"))\
                           .cache()

## Hagamos algunas preguntas a los datos para obtener conclusiones

Imaginemos que somos los dueños de una web de viajes que rastrea internet en busca de vuelos en agencias y otras páginas, los compara y recomienda el más adecuado para el aeropuerto. Junto con esta recomendación, querríamos dar también información sobre vuelos fiables y no fiables en lo que respecta a la puntualidad. Esto depende de muchos factores, como el origen y destino, duración del vuelo, hora del día, etc.

### Agrupación y agregaciones

<div class="alert alert-block alert-success">
<p><b>PREGUNTA</b>: ¿Cuáles son los vuelos (origen, destino) con mayor retraso medio? ¿Cuántos vuelos existen entre cada par de aeropuertos?</p>
<p><b>PISTA</b>: Tras hacer las agregaciones para cada pareja "Origin", "Dest" (una agregación para el retraso medio y otra para contar), aplica el método sort(F.col("avgDelay").desc()) para ordenar de forma decreciente por la nueva columna del retraso medio.
</div>

In [80]:
from pyspark.sql.functions import *

In [85]:
most_delay=cleanFlightsDF.groupBy("Origin","Dest")\
            .agg(
                    F.mean("DepDelay").alias("R_salida"),\
                    F.mean("ArrDelay").alias("R_llegada"),\
                    F.count("*").alias("count"))\
            .orderBy(col("R_salida").desc_nulls_last(),\
                     col("R_llegada").desc_nulls_last())
most_delay.show()

+------+----+------------------+------------------+-----+
|Origin|Dest|          R_salida|         R_llegada|count|
+------+----+------------------+------------------+-----+
|   RDM| MFR|             814.0|            1347.0|    2|
|   MDT| HPN|             759.0|             798.0|    1|
|   DSM| CID|             295.0|              null|    1|
|   ORD| GTF|             257.0|             212.0|    1|
|   ELM| ATL|             194.0|             169.0|    2|
|   ICT| DAY|             181.0|             210.0|    1|
|   ABE| MDT|             175.0|              null|    1|
|   ERI| ITH|             156.0|             160.0|    1|
|   DSM| PIA|             155.0|             168.0|    1|
|   CMH| HOU|             148.0|             120.0|    1|
|   CRP| BRO|             147.0|              null|    1|
|   LBB| SAF|             134.0|              null|    1|
|   AVP| SFB|             113.0|              93.0|    1|
|   YNG| PIE|             111.0|             141.0|    1|
|   PPG| HNL|1

In [4]:
from pyspark.sql import functions as F

In [86]:
combinaciones= cleanFlightsDF.withColumn("combinaciones", F.concat_ws("-","Origin","Dest"))\
                            .select("combinaciones")\
                            .groupBy("combinaciones")\
                            .agg(F.count("combinaciones").alias("n_vuelos"))
combinaciones.show(5)

+-------------+--------+
|combinaciones|n_vuelos|
+-------------+--------+
|      LGA-CVG|     925|
|      IDA-LAS|      35|
|      LAX-MTJ|      64|
|      PIE-PIA|      46|
|      AUS-CLE|      41|
+-------------+--------+
only showing top 5 rows



In [88]:
combinaciones2= cleanFlightsDF.withColumn("combinaciones", F.concat_ws("-","Origin","Dest"))\
                .groupBy("combinaciones")\
                .agg(F.avg("ArrDelay").alias("Retraso_medio"),\
                     F.count("*").alias("Number_flights"))\
                .sort(F.col("Retraso_medio").desc())
combinaciones2.show(10)

+-------------+-------------+--------------+
|combinaciones|Retraso_medio|Number_flights|
+-------------+-------------+--------------+
|      RDM-MFR|       1347.0|             2|
|      MDT-HPN|        798.0|             1|
|      ORD-GTF|        212.0|             1|
|      ICT-DAY|        210.0|             1|
|      ELM-ATL|        169.0|             2|
|      DSM-PIA|        168.0|             1|
|      ERI-ITH|        160.0|             1|
|      YNG-PIE|        141.0|             1|
|      CMH-HOU|        120.0|             1|
|      HRL-DAL|        111.0|             1|
+-------------+-------------+--------------+
only showing top 10 rows



<div class="alert alert-block alert-success">
<p><b>PREGUNTA</b>: ¿Es el avión un medio de transporte fiable? Mostrar el número de vuelos en cada categoría de retraso.</p>
En lugar de llamar agg(F.count("*")), podemos llamar a la transformación count() sobre el resultado de groupBy(), y creará
automáticamente una columna llamada "count" con los conteos para cada grupo.
<p> Ahora agrupar también por cada aeropuerto de origen, y mostrando una columna distinta por cada tipo de retraso, con el recuento. PISTA: utilizar la función pivot("colName").</p>

In [7]:
retrasos = cleanFlightsDF.select("ArrDelayCat")\
            .groupBy("ArrDelayCat")\
            .agg(F.count("ArrDelayCat"))
retrasos.show()

+-----------+------------------+
|ArrDelayCat|count(ArrDelayCat)|
+-----------+------------------+
|     Slight|            298234|
|       None|           2004727|
|       Huge|            200152|
+-----------+------------------+



In [8]:
retrasos = cleanFlightsDF.select("ArrDelayCat")\
            .groupBy("ArrDelayCat").count()
retrasos.show()

+-----------+-------+
|ArrDelayCat|  count|
+-----------+-------+
|     Slight| 298234|
|       None|2004727|
|       Huge| 200152|
+-----------+-------+



In [9]:
retrasos2 = cleanFlightsDF.groupBy("OriginCity")\
            .pivot("ArrDelayCat")\
            .agg(F.count("ArrDelayCat"))
retrasos2.show()

+------------------+----+-----+------+
|        OriginCity|Huge| None|Slight|
+------------------+----+-----+------+
|   Gainesville, FL| 139| 1147|   162|
|       Ontario, CA| 302| 5768|   771|
|     Pago Pago, TT|   4|   21|    10|
|      Richmond, VA|1013| 6686|  1055|
|        Tucson, AZ| 344| 5544|   710|
|       Medford, OR| 152| 1943|   271|
|  Myrtle Beach, SC| 262| 2335|   290|
|  Palm Springs, CA| 309| 3763|   444|
|       Redding, CA|  42|  273|    34|
|     St. Cloud, MN|   6|   54|    11|
|   Devils Lake, ND|  21|  171|    13|
|       Durango, CO| 104| 1013|   110|
|        Mobile, AL| 175| 1783|   217|
|Corpus Christi, TX| 147| 1459|   177|
|       Dubuque, IA|  39|  259|    45|
|    Huntsville, AL| 238| 2494|   287|
|     Pensacola, FL| 192| 2893|   231|
|      Columbus, GA|  40|  320|    43|
|    Fort Myers, FL|1174|12202|  1601|
|        Yakima, WA|  12|  348|    61|
+------------------+----+-----+------+
only showing top 20 rows



<div class="alert alert-block alert-success">
<p><b>PREGUNTA</b>: ¿Hay relación entre el día de la semana y el retraso a la salida o a la llegada?</p>
    <p><b>PISTA</b>: Calcula el retraso medio a la salida y a la llegada para cada día de la semana y ordena por una de ellas descendentemente.</p>
    <p> Ahora haz lo mismo para cada día pero solo con el retraso a la llegada, desagregado por cada aeropuerto de salida, utilizando la función pivot(). </p>
</div>

In [90]:
retraso_sal = cleanFlightsDF.groupBy("DayOfWeek")\
                .agg(avg("DepDelay").alias("r_salida"), avg("ArrDelay").alias("r_llegada"))\
                .orderBy(col("DayOfWeek").asc())
retraso_sal.show()

+---------+------------------+-------------------+
|DayOfWeek|          r_salida|          r_llegada|
+---------+------------------+-------------------+
|        1|10.430177708665964|  5.391113068725289|
|        2| 8.246502522185226| 2.8412409647873806|
|        3|  8.47071347600168| 3.0525338339576717|
|        4|  8.35856546210902| 2.7390527404801026|
|        5|10.220785437977693|  5.027363815430113|
|        6| 6.278199328016013|-0.5748593305876211|
|        7| 9.142161259888235| 3.2344449424598207|
+---------+------------------+-------------------+



In [93]:
retraso_lleg = flightsDF.groupBy("Origin")\
                .pivot("DayOfWeek")\
                .agg(F.mean("ArrDelay").alias("r_llegada"))\
                .sort("Origin").toPandas()
retraso_lleg

,Origin,1,2,3,4,5,6,7
0,ABE,14.953307,13.893519,15.276786,5.924370,18.183333,6.940299,10.027149
1,ABI,10.650000,16.364706,-0.547619,0.376344,4.641304,16.492063,4.606383
2,ABQ,1.442438,0.363406,1.736409,3.111111,-0.176892,-1.655721,-0.252019
3,ABR,10.571429,12.468750,6.294118,16.441176,8.529412,0.125000,9.117647
4,ABY,14.207547,3.978723,12.804348,10.645833,9.040816,29.562500,27.234043
...,...,...,...,...,...,...,...,...
351,XNA,9.592857,9.375580,6.838462,1.854015,3.877941,-0.977208,5.438861
352,YAK,-9.451613,-13.000000,-10.727273,2.848485,-9.470588,-12.088235,-10.090909
353,YKM,0.338462,-0.490909,5.666667,1.359375,6.269841,5.941176,1.603175
354,YNG,9.000000,NaN,NaN,141.000000,NaN,NaN,NaN


<div class="alert alert-block alert-info">
<p><b>LA FUNCIÓN PIVOT</b>: Puede ser interesante ver, para cada (Origin, Dest), el retraso promedio por
día de la semana. Si agrupamos por esas tres variables (Origin, Dest, DayOfWeek), nuestro resultado tendría demasiadas filas para ser fácil de visualizar (7 x 1009 ya que hay 1009 combinaciones de (Origin, DayOfWeek)). En cambio, vamos a crear 7 columnas, una por día de la semana, en nuestro resultado DF. Lo haremos utilizando una de las variables de agrupación (DayOfWeek) como <i> variable pivot</i>. Como esta variable tiene 7 valores distintos, se crearán 7 columnas nuevas. De esta manera, visualizaremos toda la información de cada combinación (Origen, Dest) condensada en una fila con 7 columnas con los 7 retrasos promedio correspondientes a ese (Origen, Dest) en cada día de la semana.
</div>

### Operaciones JOIN y de ventana

Estaría bien tener el retraso promedio de una ruta junto a cada vuelo, para que podamos ver qué vuelos tuvieron un retraso que fue superior o inferior al retraso promedio de esa ruta.

<div class="alert alert-block alert-success">
    <b> PREGUNTA </b>:
Usa el averageDelayOriginDestDF creado anteriormente, elimina la columna de conteo y luego únerlo con cleanFlightsDF, utilizando Origin y Dest como columnas de enlace. Finalmente, selecciona solo las columnas Origin, Dest, DayOfWeek, ArrDelay y avgDelay del resultado.
</div>

In [13]:
averageDelayOriginDestDF= cleanFlightsDF.groupBy("OriginCity", "DestCity")\
                .agg(avg("DepDelay").alias("r_salida"), avg("ArrDelay").alias("r_llegada"))

averageDelayOriginDestDF.show()

+----------------+--------------------+-------------------+-------------------+
|      OriginCity|            DestCity|           r_salida|          r_llegada|
+----------------+--------------------+-------------------+-------------------+
|     Detroit, MI|      Burlington, VT|  11.60135135135135|  4.913494809688581|
|  Bellingham, WA|       Las Vegas, NV| 2.1762452107279695|-3.5440613026819925|
|     Sanford, FL|    Indianapolis, IN| 14.444444444444445|  8.737373737373737|
|  South Bend, IN|  St. Petersburg, FL|-3.6964285714285716| -6.982142857142857|
|     Phoenix, AZ|        Stockton, CA| 3.9705882352941178|            2.09375|
|   Las Vegas, NV|          Moline, IL|-3.2857142857142856|-10.441176470588236|
|   Charlotte, NC|      Washington, DC|  8.382391022874407|  4.987467588591184|
|      Denver, CO|   Santa Barbara, CA|  7.178082191780822| 10.822299651567944|
|      Denver, CO|Dallas/Fort Worth...|  7.900255754475704|  4.390657084188912|
|     Phoenix, AZ|         Roswell, NM| 

Opcion 1:

In [14]:
averageDelayOriginDestDF= averageDelayOriginDestDF.withColumnRenamed("OriginCity", "Origen").withColumnRenamed("DestCity", "Dest")

In [15]:
joinExpression= (averageDelayOriginDestDF["Origen"] == cleanFlightsDF["OriginCity"]) & (averageDelayOriginDestDF["Dest"] == cleanFlightsDF["DestCity"])
joinType="left_outer"

In [16]:
DF = cleanFlightsDF.join(averageDelayOriginDestDF, joinExpression,  joinType)
DF.take(2)

[Row(Month=3, DayofMonth=24, DayOfWeek=6, FlightDate='2018-03-24', Origin='SAN', OriginCity='San Diego, CA', Dest='LAX', DestCity='Los Angeles, CA', DepTime=750, DepDelay=0.0, ArrTime=836, ArrDelay=-21.0, Cancelled=0.0, CancellationCode=None, Diverted=0.0, ActualElapsedTime=46.0, AirTime=26.0, Distance=109.0, CarrierDelay=None, WeatherDelay=None, NASDelay=None, SecurityDelay=None, LateAircraftDelay=None, ArrDelayCat='None', Origen='San Diego, CA', Dest='Los Angeles, CA', r_salida=6.496754867698452, r_llegada=3.2770843734398403),
 Row(Month=1, DayofMonth=23, DayOfWeek=2, FlightDate='2018-01-23', Origin='MBS', OriginCity='Saginaw/Bay City/Midland, MI', Dest='DTW', DestCity='Detroit, MI', DepTime=531, DepDelay=-4.0, ArrTime=612, ArrDelay=-28.0, Cancelled=0.0, CancellationCode=None, Diverted=0.0, ActualElapsedTime=41.0, AirTime=22.0, Distance=98.0, CarrierDelay=None, WeatherDelay=None, NASDelay=None, SecurityDelay=None, LateAircraftDelay=None, ArrDelayCat='None', Origen='Saginaw/Bay City/M

In [17]:
DF = DF.drop("Origen", "Dest")
DF.take(1)

[Row(Month=1, DayofMonth=19, DayOfWeek=5, FlightDate='2018-01-19', Origin='BLI', OriginCity='Bellingham, WA', DestCity='Las Vegas, NV', DepTime=1925, DepDelay=3.0, ArrTime=2154, ArrDelay=1.0, Cancelled=0.0, CancellationCode=None, Diverted=0.0, ActualElapsedTime=149.0, AirTime=136.0, Distance=954.0, CarrierDelay=None, WeatherDelay=None, NASDelay=None, SecurityDelay=None, LateAircraftDelay=None, ArrDelayCat='None', r_salida=2.1762452107279695, r_llegada=-3.5440613026819925)]

In [95]:
DF.select("OriginCity", "DestCity", "DayOfWeek", "r_salida", "r_llegada").show()

+--------------+-------------+---------+------------------+-------------------+
|    OriginCity|     DestCity|DayOfWeek|          r_salida|          r_llegada|
+--------------+-------------+---------+------------------+-------------------+
|Bellingham, WA|Las Vegas, NV|        5|2.1762452107279695|-3.5440613026819925|
|Bellingham, WA|Las Vegas, NV|        6|2.1762452107279695|-3.5440613026819925|
|Bellingham, WA|Las Vegas, NV|        7|2.1762452107279695|-3.5440613026819925|
|Bellingham, WA|Las Vegas, NV|        7|2.1762452107279695|-3.5440613026819925|
|Bellingham, WA|Las Vegas, NV|        7|2.1762452107279695|-3.5440613026819925|
|Bellingham, WA|Las Vegas, NV|        1|2.1762452107279695|-3.5440613026819925|
|Bellingham, WA|Las Vegas, NV|        1|2.1762452107279695|-3.5440613026819925|
|Bellingham, WA|Las Vegas, NV|        2|2.1762452107279695|-3.5440613026819925|
|Bellingham, WA|Las Vegas, NV|        3|2.1762452107279695|-3.5440613026819925|
|Bellingham, WA|Las Vegas, NV|        4|

Opcion 2

In [20]:
ta = cleanFlightsDF.alias("ta")
tb = averageDelayOriginDestDF.alias("tb")
df1 = cleanFlightsDF.join(averageDelayOriginDestDF, (ta.OriginCity == tb.Origen) & (ta.DestCity == tb.Dest), how="left").select(ta["*"],tb["r_llegada"],tb["r_salida"])

In [21]:
df1.select("OriginCity", "DestCity", "DayOfWeek", "r_salida", "r_llegada").show()

+--------------+-------------+---------+------------------+-------------------+
|    OriginCity|     DestCity|DayOfWeek|          r_salida|          r_llegada|
+--------------+-------------+---------+------------------+-------------------+
|Bellingham, WA|Las Vegas, NV|        5|2.1762452107279695|-3.5440613026819925|
|Bellingham, WA|Las Vegas, NV|        6|2.1762452107279695|-3.5440613026819925|
|Bellingham, WA|Las Vegas, NV|        7|2.1762452107279695|-3.5440613026819925|
|Bellingham, WA|Las Vegas, NV|        7|2.1762452107279695|-3.5440613026819925|
|Bellingham, WA|Las Vegas, NV|        7|2.1762452107279695|-3.5440613026819925|
|Bellingham, WA|Las Vegas, NV|        1|2.1762452107279695|-3.5440613026819925|
|Bellingham, WA|Las Vegas, NV|        1|2.1762452107279695|-3.5440613026819925|
|Bellingham, WA|Las Vegas, NV|        2|2.1762452107279695|-3.5440613026819925|
|Bellingham, WA|Las Vegas, NV|        3|2.1762452107279695|-3.5440613026819925|
|Bellingham, WA|Las Vegas, NV|        4|

<div class="alert alert-block alert-info">
    <p><b>BONUS (OPCIONAL)</b>: crear una nueva columna <i>belowAverage</i> que tenga valor True si ArrDelay es menor que el avgDelay de esa ruta, y False en caso contrario. No utilizar la función when() sino el operador de comparación directamente entre columnas, la cual devolverá una columna booleana.
</div>

In [29]:
b_avg = df1.withColumn("belowAverage", col("ArrDelay")<col("r_llegada"))

In [31]:
b_avg.select("OriginCity", "DestCity", "ArrDelay", "r_llegada", "belowAverage").show()

+--------------+-------------+--------+-------------------+------------+
|    OriginCity|     DestCity|ArrDelay|          r_llegada|belowAverage|
+--------------+-------------+--------+-------------------+------------+
|Bellingham, WA|Las Vegas, NV|     1.0|-3.5440613026819925|       false|
|Bellingham, WA|Las Vegas, NV|    -6.0|-3.5440613026819925|        true|
|Bellingham, WA|Las Vegas, NV|   -11.0|-3.5440613026819925|        true|
|Bellingham, WA|Las Vegas, NV|   -13.0|-3.5440613026819925|        true|
|Bellingham, WA|Las Vegas, NV|    -8.0|-3.5440613026819925|        true|
|Bellingham, WA|Las Vegas, NV|   -21.0|-3.5440613026819925|        true|
|Bellingham, WA|Las Vegas, NV|   -16.0|-3.5440613026819925|        true|
|Bellingham, WA|Las Vegas, NV|    -7.0|-3.5440613026819925|        true|
|Bellingham, WA|Las Vegas, NV|     0.0|-3.5440613026819925|       false|
|Bellingham, WA|Las Vegas, NV|   -10.0|-3.5440613026819925|        true|
|Bellingham, WA|Las Vegas, NV|   -13.0|-3.544061302

**PREGUNTA**: repetir la operación utilizando funciones de ventana, sin usar `join`.

<div class="alert alert-block alert-success">
<b> PREGUNTA </b>: Vamos a construir otro DF con información sobre los aeropuertos (en una situación real, tendríamos otra tabla en la base de datos como la tabla de la entidad Aeropuerto). Sin embargo, solo tenemos información sobre algunos aeropuertos. Nos gustaría agregar esta información a cleanFlightsDF como nuevas columnas, teniendo en cuenta que queremos que la información del aeropuerto coincida con el aeropuerto de origen de flightsDF. Utilizar la operación de unión adecuada para asegurarse de que no se perderá ninguna de las filas existentes de cleanFlightsDF después de la unión.
</div>

In [36]:
airportsDF = spark.createDataFrame([
    ("JFK", "John F. Kennedy International Airport", 1948),
    ("LIT", "Little Rock National Airport", 1931),
    ("SEA", "Seattle-Tacoma International Airport", 1949),
], ["IATA", "FullName", "Year"])

In [38]:
joinExpression= (airportsDF["IATA"]==cleanFlightsDF["Origin"])
joinType="left_outer"

In [41]:
ta = cleanFlightsDF.alias("ta")
tb = airportsDF.alias("tb")
joinedFlightsDF =  cleanFlightsDF.join(airportsDF, joinExpression, joinType).select(ta["*"],tb["FullName"],tb["Year"])

In [62]:
joinedFlightsDF.take(1)

[Row(Month=1, DayofMonth=2, DayOfWeek=2, FlightDate='2018-01-02', Origin='BGM', OriginCity='Binghamton, NY', Dest='DTW', DestCity='Detroit, MI', DepTime=552, DepDelay=-9.0, ArrTime=747, ArrDelay=-2.0, Cancelled=0.0, CancellationCode=None, Diverted=0.0, ActualElapsedTime=115.0, AirTime=69.0, Distance=378.0, CarrierDelay=None, WeatherDelay=None, NASDelay=None, SecurityDelay=None, LateAircraftDelay=None, ArrDelayCat='None', FullName=None, Year=None)]

In [74]:
# PREGUNTA: mostrar algunas filas donde FullName no sea null
joinedFlightsDF.select("FullName", "Origin", "Dest").where(col("FullName") != "UNKOWN").show()

+--------------------+------+----+
|            FullName|Origin|Dest|
+--------------------+------+----+
|Seattle-Tacoma In...|   SEA| JFK|
|Seattle-Tacoma In...|   SEA| LGB|
|Seattle-Tacoma In...|   SEA| BOS|
|Seattle-Tacoma In...|   SEA| BOS|
|Seattle-Tacoma In...|   SEA| LGB|
|Seattle-Tacoma In...|   SEA| JFK|
|Seattle-Tacoma In...|   SEA| LGB|
|Seattle-Tacoma In...|   SEA| BOS|
|Seattle-Tacoma In...|   SEA| BOS|
|Seattle-Tacoma In...|   SEA| LGB|
|Seattle-Tacoma In...|   SEA| JFK|
|Seattle-Tacoma In...|   SEA| LGB|
|Seattle-Tacoma In...|   SEA| BOS|
|Seattle-Tacoma In...|   SEA| BOS|
|Seattle-Tacoma In...|   SEA| LGB|
|Seattle-Tacoma In...|   SEA| JFK|
|Seattle-Tacoma In...|   SEA| LGB|
|Seattle-Tacoma In...|   SEA| BOS|
|Seattle-Tacoma In...|   SEA| BOS|
|Seattle-Tacoma In...|   SEA| LGB|
+--------------------+------+----+
only showing top 20 rows



## User-defined functions (UDFs)

Vamos a construir un UDF para convertir millas a kilómetros. Ten en cuenta que esto podría hacerse fácilmente multiplicando directamente la columna de millas por 1.6 (y sería mucho más eficiente), ya que Spark permite el producto entre una columna y un número. En todos los casos en los que Spark proporciona funciones integradas para realizar una tarea (como esta), debes usar esas funciones y no una UDF. Las UDF deben emplearse solo cuando no hay otra opción.

La razón es que las funciones integradas de Spark están optimizadas y Catalyst, el optimizador automático de código integrado en Spark, puede optimizarlo aún más. Sin embargo, las UDF son una caja negra para Catalyst y su contenido no se optimizará, y por lo tanto, generalmente son mucho más lentas.

In [75]:
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

# Primer paso: crear una función de Python que reciba UN número y lo multiplique por 1.6
def milesToKm(miles):
    return miles*1.6

# Vamos a probarla
print(milesToKm(5)) # 5 millas a km: 8 km

# Segundo paso: crear un objeto UDF que envuelva a nuestra función. 
# Hay que especificar el tipo de dato que devuelve nuestra función
udfMilesToKm = F.udf(milesToKm, DoubleType())

# Con esto, Spark será capaz de llamar a nuestra función milesToKm sobre cada uno de los valores de una columna numérica.
# Spark enviará el código de nuestra función a los executors a través de la red, y cada executor la ejecutará sobre las
# particiones (una por una) que estén en ese executor

# Tercer paso: vamos a probar la UDF añadiendo una nueva columna con el resultado de la conversión
flightsWithKm = cleanFlightsDF.withColumn("DistKm", udfMilesToKm(F.col("Distance")))

flightsWithKm.select("Origin", "Dest", "Distance", "DistKM")\
             .distinct()\
             .show(5)

8.0
+------+----+--------+------------------+
|Origin|Dest|Distance|            DistKM|
+------+----+--------+------------------+
|   CLE| JFK|   425.0|             680.0|
|   MCO| PSE|  1179.0|            1886.4|
|   FLL| DTW|  1127.0|            1803.2|
|   LAX| MTJ|   666.0|1065.6000000000001|
|   AZA| MLI|  1288.0|            2060.8|
+------+----+--------+------------------+
only showing top 5 rows



<div class="alert alert-block alert-info">
<p><b>BONUS</b>: Crea tu propia UDF que convierta DayOfWeek en una cadena.
Puedes hacerlo creando una función de Python que reciba un número entero y devuelva el día de la semana,
simplemente leyendo desde un vector de cadenas de longitud 7 el valor en la posición indicada por el argumento entero. Para la UDF, recuerda que tu función devuelve un StringType(). Finalmente, prueba tu UDF creando una nueva columna "DayOfWeekString".
</div>

In [78]:
from pyspark.sql.types import StringType

# Primer paso: creamos una función de python que convierte un número entero en el día de la semana como cadena
def dayOfWeekToString(dayInteger):
    # En nuestros datos Monday es 1 pero las listas de python empiezan en el 0 y 
    # queremos usar el dayInteger como índice del vector
    daysOfWeek = ["", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    return daysOfWeek[dayInteger]
    
print(dayOfWeekToString(2))

# Segundo paso: ajustamos nuestra función con un Spark UDF para que Spark pueda invocarlo en cada valor de una columna completa
# De esta manera, Spark puede enviar nuestra función a los ejecutores, que eventualmente ejecutarán la función en las particiones
# de los datos que tiene cada ejecutor
dayOfWeekStringUDF = F.udf(dayOfWeekToString, StringType())

# Tercer paso: intentemos nuestro UDF agregando una nueva columna que resulta de transformar (a través del UDF) el
# columna existente DayOfWeek
flightsWithDayOfWeekStr = cleanFlightsDF.withColumn("DayOfWeekString", dayOfWeekStringUDF(F.col("DayOfWeek")))

flightsWithDayOfWeekStr.select("Origin", "Dest", "DayOfWeek", "DayOfWeekString")\
                       .distinct()\
                       .show()

Tuesday
+------+----+---------+---------------+
|Origin|Dest|DayOfWeek|DayOfWeekString|
+------+----+---------+---------------+
|   BQK| ATL|        4|       Thursday|
|   CVG| PHL|        3|      Wednesday|
|   DTW| DFW|        5|         Friday|
|   SEA| JFK|        2|        Tuesday|
|   JAX| JFK|        2|        Tuesday|
|   RDU| BOS|        3|      Wednesday|
|   SEA| BOS|        3|      Wednesday|
|   AUS| FLL|        3|      Wednesday|
|   JFK| LAS|        5|         Friday|
|   SLC| BOS|        6|       Saturday|
|   BOS| HOU|        6|       Saturday|
|   BDL| MCO|        7|         Sunday|
|   SJU| TPA|        7|         Sunday|
|   PGD| TYS|        6|       Saturday|
|   PIE| CVG|        6|       Saturday|
|   ABE| SFB|        7|         Sunday|
|   LAS| BIS|        7|         Sunday|
|   ROC| PGD|        1|         Monday|
|   EWR| CVG|        1|         Monday|
|   CVG| SFB|        1|         Monday|
+------+----+---------+---------------+
only showing top 20 rows

